In [6]:
# I'M GOING TO TRY TO FINISH THIS THIS WEEKEND
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
random.seed(134)

PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32

Perform tuning over at least two of the following hyperparameters:
• Varying the size of the hidden dimension of the CNN and RNN
• Varying the kernel size of the CNN
• Experiment with different ways of interacting the two encoded sentences
(concatenation, element-wise multiplication, etc)
• Regularization (e.g. weight decay, dropout)

For each mode of hyperparameter tuning, report the training and validation losses and accuracies (in plotted curves)


In [7]:
train_df = pd.read_csv('hw2_data/snli_train.tsv', sep='\t')
val_df = pd.read_csv('hw2_data/snli_val.tsv', sep='\t')

In [8]:
train_df.iloc[0]

sentence1    A young girl in a pink shirt sitting on a dock...
sentence2    A young girl watching the sunset over the water .
label                                                  neutral
Name: 0, dtype: object

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# Import tokenizer
stopset = set(stopwords.words('english'))

In [ ]:
def tokenize(text): 
    # punctuation might actually be important here. 
    tokens=word_tokenize(str(text))
    tokens = [w for w in tokens if not w in stopset]
    return tokens

In [ ]:
text = tokenize(train_df

In [ ]:
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(all_train_tokens)


In [ ]:
# THE INITIAL MODEL 
class RNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size):
        # RNN Accepts the following hyperparams:
        # emb_size: Embedding Size
        # hidden_size: Hidden Size of layer in RNN
        # num_layers: number of layers in RNN
        # num_classes: number of output classes
        # vocab_size: vocabulary size
        super(RNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.rnn = nn.RNN(emb_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, num_classes)

    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        hidden = torch.randn(self.num_layers, batch_size, self.hidden_size)

        return hidden

    def forward(self, x, lengths):
        # reset hidden state

        batch_size, seq_len = x.size()

        self.hidden = self.init_hidden(batch_size)

        # get embedding of characters
        pdb.set_trace()
        embed = self.embedding(x)
        # pack padded sequence (which means that, given a sequence that is padded, remove all the 0s so you only get)
        # an array of sum(length), word_embedding_Size (also flatten)
        embed = torch.nn.utils.rnn.pack_padded_sequence(embed, lengths.numpy(), batch_first=True)
        # fprop though RNN
        rnn_out, self.hidden = self.rnn(embed, self.hidden)
        # undo packing, this basically inserts back the zeroes
        rnn_out, _ = torch.nn.utils.rnn.pad_packed_sequence(rnn_out, batch_first=True)
        # sum hidden activations of RNN across time
        rnn_out = torch.sum(rnn_out, dim=1)

        logits = self.linear(rnn_out)
        return logits